In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import numpy as np
import seaborn as sn
import pickle
import matplotlib.pyplot as plt



In [2]:
final_data = pd.read_csv("join.csv")

In [3]:
features = final_data[
    [
        "Country",
        "Item",
        "Average Rainfall (mm/year)",
        "Pesticides (Tonnes)",
        "Average Temperature",
    ]
].copy()


response = final_data["Yield (hg/ha)"].copy()


In [4]:
# main test split
X_train, X_test, y_train, y_test = train_test_split(
    features,
    response,
    test_size=0.2,
    random_state=42,
    stratify=features['Item']
)

In [5]:
class Country_modifier(BaseEstimator, TransformerMixin):
    def _init_(self):
        self.country_counts = {}
        return

    def fit(self, X, y=None):
        self.country_counts = dict(X.Country.value_counts())
        return self

    def transform(self, X, y=None):
        X["Country"] = X["Country"].apply(
            lambda x: x if self.country_counts[x] >= 200 else "Other"
        )

        return X

In [6]:
ct = ColumnTransformer(
    [
        ("One Hot Encoder", OneHotEncoder(handle_unknown="error"), ["Country", "Item"]),
        (
            "Standard Scaler",
            StandardScaler(),
            [
                "Average Rainfall (mm/year)",
                "Pesticides (Tonnes)",
                "Average Temperature",
            ],
        ),
    ]
)


In [7]:
final_pipeline = Pipeline(steps=[
    ('Country Modifier', Country_modifier()),
    ('One hot Encode and Standardize', ct),
    ('RF_model', RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42))

])

In [8]:
final_pipeline.fit(X_train, y_train)


C:\Users\bilal\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Pipeline(steps=[('Country Modifier', Country_modifier()),
                ('One hot Encode and Standardize',
                 ColumnTransformer(transformers=[('One Hot Encoder',
                                                  OneHotEncoder(),
                                                  ['Country', 'Item']),
                                                 ('Standard Scaler',
                                                  StandardScaler(),
                                                  ['Average Rainfall (mm/year)',
                                                   'Pesticides (Tonnes)',
                                                   'Average Temperature'])])),
                ('RF_model',
                 RandomForestRegressor(max_depth=10, n_estimators=200,
                                       random_state=42))])

In [9]:
with open ('model_pickle', 'wb') as f:
    pickle.dump(final_pipeline,f)

In [10]:
score = final_pipeline.predict(X_test)
r2_score(y_test, score)

C:\Users\bilal\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


0.9070764399555149